In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import time
from tqdm import tqdm

<IPython.core.display.Javascript object>

In [3]:
from steamship import Steamship, PluginInstance, File

from steamship import Block, Tag
from steamship.base.tasks import TaskState

<IPython.core.display.Javascript object>

In [4]:
client = Steamship(profile="prod", workspace="podcasts")


def get_steamship_client():
    return client

<IPython.core.display.Javascript object>

In [5]:
FILE_IMPORTER_HANDLE = "youtube-file-importer"

file_importer = PluginInstance.create(
    client,
    plugin_handle=FILE_IMPORTER_HANDLE,
    config={},
).data
print("importer", file_importer.id)

importer B56D535F-2E5B-4532-B467-B4B5E40EFCC6


<IPython.core.display.Javascript object>

In [6]:
BLOCKIFIER_HANDLE = "s2t-blockifier-default"

s2t_blockifier = PluginInstance.create(
    client,
    plugin_handle=BLOCKIFIER_HANDLE,
    config={},
).data
print("blockifier", s2t_blockifier.id)

blockifier 3326998E-064D-41B2-BF2D-077BBED26A03


<IPython.core.display.Javascript object>

In [7]:
podcasts = [
    ("https://youtu.be/uXyS-74sRTE", "Louis Theroux"),
    ("https://youtu.be/vGc4mg5pul4", "Neil deGrasse Tyson"),
    ("https://youtu.be/rY5jCvRHEFk", "Russell Brand"),
    ("https://youtu.be/Rxmw9eizOAo", "Graham Hancock"),
]

<IPython.core.display.Javascript object>

In [8]:
file_futures = []
for podcast_url, podcast_guest in tqdm(podcasts):
    file = File.create(
        client, plugin_instance=file_importer.handle, url=podcast_url
    ).data
    file_futures.append(
        (
            file.blockify(plugin_instance=s2t_blockifier.handle),
            file,
            podcast_url,
            podcast_guest,
        )
    )

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:58<00:00, 14.72s/it]


<IPython.core.display.Javascript object>

In [10]:
skipped_urls = []
while len(skipped_urls) < len(file_futures):
    print(".", end="")
    for file_future, file, podcast_url, podcast_guest in file_futures:
        if podcast_url not in skipped_urls:
            try:
                file_future.wait(max_timeout_s=2, retry_delay_s=1)
            except:
                pass
            if file_future.task.state in (TaskState.succeeded, TaskState.failed):
                print(file.id, file_future.task.state)
                if file_future.task.state == TaskState.succeeded:
                    try:
                        skipped_urls.append(podcast_url)
                        file = file.refresh().data
                        f = File.create(
                            client,
                            tags=[
                                Tag.CreateRequest(kind="youtube_url", name=podcast_url),
                                Tag.CreateRequest(kind="guest", name=podcast_guest),
                            ],
                            blocks=file.blocks,
                        ).data
                        tags = f.blocks[0].tags
                        print(len(tags), {tag.kind for tag in tags})
                        file.delete()
                    except Exception:
                        pass

    time.sleep(5)

.BC45737C-7AB9-4D0A-991C-4729CE493636 succeeded
CDA7F09F-11B0-4C36-9AB8-7EC2F4375EA0 failed
DEC50DA6-A333-4E74-B95B-D66DC7D908DC succeeded


Unable to find object id=DEC50DA6-A333-4E74-B95B-D66DC7D908DC handle=?
ERROR:root:Client received error from server: [ObjectNotFound]
Unable to find object id=DEC50DA6-A333-4E74-B95B-D66DC7D908DC handle=?


7E2C9BA0-452D-4D8F-BDF3-0FEB6B3D6BF8 succeeded


Unable to find object id=7E2C9BA0-452D-4D8F-BDF3-0FEB6B3D6BF8 handle=?
ERROR:root:Client received error from server: [ObjectNotFound]
Unable to find object id=7E2C9BA0-452D-4D8F-BDF3-0FEB6B3D6BF8 handle=?


.CDA7F09F-11B0-4C36-9AB8-7EC2F4375EA0 failed
.CDA7F09F-11B0-4C36-9AB8-7EC2F4375EA0 failed
.CDA7F09F-11B0-4C36-9AB8-7EC2F4375EA0 failed
.CDA7F09F-11B0-4C36-9AB8-7EC2F4375EA0 failed
.CDA7F09F-11B0-4C36-9AB8-7EC2F4375EA0 failed
.CDA7F09F-11B0-4C36-9AB8-7EC2F4375EA0 failed
.CDA7F09F-11B0-4C36-9AB8-7EC2F4375EA0 failed
.CDA7F09F-11B0-4C36-9AB8-7EC2F4375EA0 failed
.CDA7F09F-11B0-4C36-9AB8-7EC2F4375EA0 failed
.CDA7F09F-11B0-4C36-9AB8-7EC2F4375EA0 failed
.CDA7F09F-11B0-4C36-9AB8-7EC2F4375EA0 failed
.CDA7F09F-11B0-4C36-9AB8-7EC2F4375EA0 failed
.CDA7F09F-11B0-4C36-9AB8-7EC2F4375EA0 failed
.CDA7F09F-11B0-4C36-9AB8-7EC2F4375EA0 failed


KeyboardInterrupt: 

<IPython.core.display.Javascript object>